In [1]:
import boto3
import json
import requests
from concurrent.futures import ThreadPoolExecutor
import concurrent
from typing import Generator, Dict, Any
import itertools
from urllib.parse import urlparse, parse_qs
import os
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import pandas as pd


boto3.setup_default_session(profile_name='r2')
s3_client = boto3.client('s3')

image_bucket = "salad-benchmark-public-assets"
bucket_domain = "https://salad-benchmark-assets.download"

In [2]:
def list_all_file_urls(bucket: str, prefix: str) -> Generator[str, None, None]:
    paginator = s3_client.get_paginator('list_objects_v2')
    page_iterator = paginator.paginate(Bucket=bucket, Prefix=prefix)
    for page in page_iterator:
        for content in page['Contents']:
            yield f"{bucket_domain}/{content['Key']}"
            
def create_session(max_retries=3, backoff_factor=0.3):
    """
    Create a requests session with retry strategy.
    :param max_retries: Maximum number of retries for each request.
    :param backoff_factor: A backoff factor to apply between attempts.
    :param status_forcelist: A set of HTTP status codes that we should force a retry on.
    :return: A requests session object.
    """
    session = requests.Session()
    retries = Retry(total=max_retries,
                    read=max_retries,
                    connect=max_retries,
                    backoff_factor=backoff_factor,
                    status_forcelist=[i for i in range(500, 600)],
                    respect_retry_after_header=True
                    )
    adapter = HTTPAdapter(max_retries=retries)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

# Usage
session = create_session()
            
num_requests = 0
num_responses = 0
def http_worker(request_params: Dict[str, Any]) -> requests.Response:
    """ Function to make an HTTP request """
    global num_requests, num_responses
    num_requests += 1
    print(f"\r{num_requests} submitted | {num_responses} responses", end="", flush=True)
    response = session.request(**request_params)
    num_responses += 1
    return response
      
def chunk_list(lst, chunk_size):
    """Yield successive chunks of chunk_size from lst."""
    for i in range(0, len(lst), chunk_size):
        yield lst[i:i + chunk_size]

def fetch_responses(requests_generator: Generator[Dict[str, Any], None, None], 
                    pool_size: int = 5) -> Generator[requests.Response, None, None]:
    """ Function to manage a pool of HTTP workers """
    
    with ThreadPoolExecutor(max_workers=pool_size) as executor:
        # Chunk the requests into groups of (pool_size * 10)
        for chunk in chunk_list(requests_generator, pool_size * 10):
            futures = [executor.submit(http_worker, params) for params in chunk]
            for future in concurrent.futures.as_completed(futures):
                yield future.result()

In [3]:
image_tagging_api = "https://orange-splitpea-kunklrrfjwnyterv.salad.cloud"

def all_requests():
    for url in list_all_file_urls(image_bucket, "coco2017/train2017/"):
        yield {
          "method": "GET",
          "url": f"{image_tagging_api}/tag",
          "params": {"url": url}
        }
    for url in list_all_file_urls(image_bucket, "ava/images/"):
        yield {
          "method": "GET",
          "url": f"{image_tagging_api}/tag",
          "params": {"url": url}
        }
        
def get_image_url_from_response(response: requests.Response) -> str:
    return parse_qs(urlparse(response.request.url).query)["url"][0]
    
    
def get_rows(request_generator):
    for response in fetch_responses(request_generator, pool_size=50):
        if response.status_code != 200:
            raise Exception(f"Error: {response.status_code} {response.reason}")
        body = response.json()
        body["image_url"] = get_image_url_from_response(response)
        salad_headers = [header for header in response.headers if header in ["x-gpu-name", "x-salad-machine-id", "x-salad-container-group-id", "x-inference-time", "x-image-download-time"]]
        for header in salad_headers:
            body[header] = response.headers[header]
        yield body

In [4]:
request_list_cache_file = "all_requests.json"

if os.path.exists(request_list_cache_file):
    with open(request_list_cache_file) as f:
        list_of_all_requests = json.load(f)
else:
    list_of_all_requests = list(all_requests())
    with open("all_requests.json", "w") as f:
        json.dump(list_of_all_requests, f)
print(f"Total number of requests: {len(list_of_all_requests)}")


Total number of requests: 373795


In [5]:
row_file = "all_rows.jsonl"
if os.path.exists(row_file):
    with open(row_file) as f:
        rows = [json.loads(line) for line in f]
else:
    rows = []
    with open(row_file, "w") as f:
        for row in get_rows(list_of_all_requests):
            f.write(json.dumps(row) + "\n")
            rows.append(row)
            
print(f"Total number of rows: {len(rows)}")


Total number of rows: 144485


In [6]:
df = pd.DataFrame(rows)
df

,tags,image_url,x-salad-machine-id,x-salad-container-group-id,x-gpu-name,x-inference-time,x-image-download-time
0,[Close-up],https://salad-benchmark-assets.download/coco20...,10a8f298-321a-4a55-a75f-d52b81aede93,1897e5ba-ae3f-4322-939f-fc3b4fdb8507,NVIDIA GeForce RTX 3060 Ti,0.2417,0.1154
1,"[Close-up, Plant stem]",https://salad-benchmark-assets.download/coco20...,01031b4a-4424-9551-97aa-4a74c5ed172f,1897e5ba-ae3f-4322-939f-fc3b4fdb8507,NVIDIA GeForce RTX 4070,0.0311,0.2113
2,"[Close-up, Leaf vegetable, Vegetarian food]",https://salad-benchmark-assets.download/coco20...,378781b8-1691-f95e-9454-02e23449a47b,1897e5ba-ae3f-4322-939f-fc3b4fdb8507,NVIDIA GeForce RTX 4070,0.3527,0.1609
3,"[Black-and-white, Close-up, Digital camera]",https://salad-benchmark-assets.download/coco20...,ad04804a-eab8-1159-b566-970756ac0b26,1897e5ba-ae3f-4322-939f-fc3b4fdb8507,NVIDIA GeForce RTX 4070,0.0328,0.3307
4,[Jeans],https://salad-benchmark-assets.download/coco20...,4276c021-3b95-5e50-9d8e-0924b882ee5d,1897e5ba-ae3f-4322-939f-fc3b4fdb8507,NVIDIA GeForce RTX 3070,0.2407,0.2163
...,...,...,...,...,...,...,...
144480,"[Arthropod, Black-and-white, Close-up, Plant s...",https://salad-benchmark-assets.download/ava/im...,c380c652-b346-5e58-8ef5-c637f74b7d30,1897e5ba-ae3f-4322-939f-fc3b4fdb8507,NVIDIA GeForce RTX 3060,0.2322,0.4679
144481,[Close-up],https://salad-benchmark-assets.download/ava/im...,1185b629-585e-2b50-a136-db9ff806799b,1897e5ba-ae3f-4322-939f-fc3b4fdb8507,NVIDIA GeForce RTX 4070 Ti,0.0365,0.7660
144482,[Close-up],https://salad-benchmark-assets.download/ava/im...,77a3060e-5844-fc51-9547-37ea0c733ae1,1897e5ba-ae3f-4322-939f-fc3b4fdb8507,NVIDIA GeForce RTX 4080,0.2103,0.5672
144483,"[Close-up, Cobblestone, Plant stem]",https://salad-benchmark-assets.download/ava/im...,c8206b04-901d-8753-a5f8-4132dd4ce440,1897e5ba-ae3f-4322-939f-fc3b4fdb8507,NVIDIA GeForce RTX 3060,0.3511,1.4748


In [7]:
num_machines = len(df["x-salad-machine-id"].unique())
print(f"Number of unique machines: {num_machines}")

Number of unique machines: 167


In [8]:
gpu_types = sorted(df["x-gpu-name"].unique())
num_gpu_types = len(gpu_types)
print(f"Number of unique GPU types: {num_gpu_types}")
print(f"GPU types: {gpu_types}")

Number of unique GPU types: 20
GPU types: ['NVIDIA GeForce RTX 2080', 'NVIDIA GeForce RTX 2080 SUPER', 'NVIDIA GeForce RTX 2080 Ti', 'NVIDIA GeForce RTX 3060', 'NVIDIA GeForce RTX 3060 Ti', 'NVIDIA GeForce RTX 3070', 'NVIDIA GeForce RTX 3070 Laptop GPU', 'NVIDIA GeForce RTX 3070 Ti', 'NVIDIA GeForce RTX 3080', 'NVIDIA GeForce RTX 3080 Laptop GPU', 'NVIDIA GeForce RTX 3080 Ti', 'NVIDIA GeForce RTX 3090', 'NVIDIA GeForce RTX 4060', 'NVIDIA GeForce RTX 4060 Laptop GPU', 'NVIDIA GeForce RTX 4060 Ti', 'NVIDIA GeForce RTX 4070', 'NVIDIA GeForce RTX 4070 Laptop GPU', 'NVIDIA GeForce RTX 4070 Ti', 'NVIDIA GeForce RTX 4080', 'NVIDIA GeForce RTX 4090']


In [18]:
import plotly.express as px

# Getting the value counts
gpu_type_counts = df["x-gpu-name"].value_counts()

# Convert the Series to DataFrame and reset the index
gpu_type_counts_df = gpu_type_counts.reset_index()
gpu_type_counts_df.columns = ['x-gpu-name', 'count']

# Removing "NVIDIA GeForce" from the GPU names
gpu_type_counts_df['x-gpu-name'] = gpu_type_counts_df['x-gpu-name'].str.replace('NVIDIA GeForce ', '')

# Sort the DataFrame by 'x-gpu-name'
gpu_type_counts_df = gpu_type_counts_df.sort_values(by='x-gpu-name')

# Create the bar graph
fig = px.bar(gpu_type_counts_df, x='x-gpu-name', y='count', title="Number of Images Tagged by GPU")

# Show the figure
fig.show()

fig.update_layout(
    title_text=f"Number of Images Tagged by GPU",
    title_font=dict(size=32),
    xaxis_title="GPU",
    xaxis_title_font=dict(size=24),
    yaxis_title="Images Tagged",
    yaxis_title_font=dict(size=24),
    xaxis=dict(tickfont=dict(size=18)),
    yaxis=dict(tickfont=dict(size=18)),
)
fig.write_image("gpu_type_counts.jpg", width=1920, height=768)
